In [ ]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('vader_lexicon')

In [4]:
import requests
import re
import pandas as pd
import numpy as np

import time
import random
import csv

from textblob import TextBlob
from bs4 import BeautifulSoup

import torch
import transformers
from newsapi import NewsApiClient
from datetime import datetime, timedelta

In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC

In [6]:
# from nltk.sentiment.vader import SentimentIntensityAnalyzer
# analyzer = SentimentIntensityAnalyzer()
# text = "Bitcoin is on the rise, and investors are bullish on its future prospects."
# Calculate sentiment scores using VADER
# scores = analyzer.polarity_scores(text)
# print(scores)

In [7]:
API_KEY ='a2b6e49ae229461d9da8a2b34cfc2f02'
newsapi = NewsApiClient(api_key=API_KEY)

# Calculate the date range for the past week
today = datetime.now()
last_week = today - timedelta(days=7)
from_date = last_week.strftime('%Y-%m-%d')
to_date = today.strftime('%Y-%m-%d')

In [8]:
all_articles = newsapi.get_everything(q='investment',
                                    from_param=from_date,
                                    to=to_date,
                                    language='en',
                                    sort_by='publishedAt')

In [4]:
url = 'https://newsapi.org/v2/top-headlines'
params = {
    'country': 'id',
    'category': 'business',
    'apiKey': API_KEY,
    'from': from_date,
    'to': to_date,
    'sortBy': 'publishedAt',
}
response = requests.get(url, params=params)

if response.status_code == 200:
    data = response.json()
    articles = data.get('articles', [])
    articles_dict = {
        'titles': [],
        'descriptions': [],
        'urls': []
    }
    for article in articles:
        articles_dict['titles'].append(article['title'])
        articles_dict['descriptions'].append(article['description'])
        articles_dict['urls'].append(article['url'])
else:
    print(f"Failed to fetch news: {response.status_code}")

In [ ]:
articles_dict['urls']

In [ ]:
# urls = {
#     'BEISectoralSummary': 'https://www.idx.co.id/primary/StockData/GetIndexIC',
#     'BEIIndexSummary': 'https://www.idx.co.id/primary/StockData/GetConstituent',
# }

article_links = [
    'https://www.idx.co.id/id/produk/saham',
    'https://www.idx.co.id/id/produk/surat-utang-obligasi',
    'https://www.idx.co.id/id/produk/reksa-dana',
    'https://www.idx.co.id/id/produk/exchange-traded-fund-etf',
    'https://www.idx.co.id/id/produk/dire-dinfra',
    'https://www.idx.co.id/id/produk/derivatif',
    'https://www.idx.co.id/id/produk/structured-warrant-sw',
    'https://www.idx.co.id/id/produk/indeks',
    'https://www.idx.co.id/id/produk-layanan/jam-dan-mekanisme-perdagangan',
    'https://www.idx.co.id/id/investhub/perpajakan',
    'https://www.idx.co.id/id/produk/layanan-data-bei'
]

In [ ]:
def create_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--headless=new")
    options.binary_location = r"C:\Program Files\Google\Chrome\Application\chrome.exe"

    # Set user agent and other headers to mimic a real browser
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.5672.63 Safari/537.36")
    options.add_argument("accept-language=en-US,en;q=0.9")
    options.add_argument("referer=https://www.google.com/")

    # Disable features that indicate headless browsing
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_experimental_option('excludeSwitches', ['enable-automation'])
    options.add_experimental_option('useAutomationExtension', False)

    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager(driver_version="125.0.6422.113").install()), options=options)
    return driver

def human_delay():
    time.sleep(int(2 + (random.random() * 3.5)))

def scrape_google_search_links(urls):
    driver = create_driver()

    try:
        for url in urls:
            driver.get(url)
            WebDriverWait(driver, timeout=10).until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
            human_delay()
            # Find all search result links
            search_results = driver.find_elements(By.CSS_SELECTOR, 'a[href^="http"]')
            for link in search_results:
                href = link.get_attribute('href')
                if not href.startswith('https://www.google.com/') and not href.startswith('https://www.gstatic.com/') and not href.startswith('https://support.google.com/') and not href.startswith('https://www.google.co.id/') and not href.startswith('https://accounts.google.com/') and not href.startswith('https://maps.google.com/') and not href.startswith('https://policies.google.com') and not href.startswith('https://www.youtube.com'):
                    article_links.append(href)
            human_delay()

    except Exception as e:
        print(f"An error occurred: {e}")

    finally:
        driver.quit()
        return article_links

# Example usage
urls = [
    "https://www.google.com/search?q=saham",
    "https://www.google.com/search?q=investasi"
]

searched_links = scrape_google_search_links(urls)
print(searched_links)

In [ ]:
print(article_links)

In [ ]:
def scrape_article_urls(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }

    response = requests.get(url, headers=headers)
    article_urls = []
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")

        # Find all article links on the search results page
        articles = soup.find_all("article")
        for article in articles:
            link_tag = article.find("a")
            if link_tag and 'href' in link_tag.attrs:
                article_url = link_tag['href']
                # Ensure the link is complete
                if not article_url.startswith("http"):
                    article_url = "https://www.cnnindonesia.com" + article_url
                article_urls.append(article_url)
                print(f"Article URL: {article_url}")
    else:
        print("Failed to retrieve the webpage. Status code:", response.status_code)
    
    return article_urls


base_url = "https://www.cnnindonesia.com/search/?query="
queries = ['investasi', 'saham', 'bisnis', 'reksadana']
    
all_article_urls = []
for query in queries:
    search_url = f"{base_url}{query}"
    print(f"Scraping article URLs for query: {query}")
    article_urls = scrape_article_urls(search_url)
    all_article_urls.extend(article_urls)
    
    # Print all collected article URLs
print("\nAll collected article URLs:")
for url in all_article_urls:
    print(url)

In [ ]:
combined_link = sorted(set(article_links + all_article_urls))

# Open a file in write mode
with open("combined_link.txt", "w") as file:
    # Write each sorted unique item to the file
    file.writelines(f"{item}\n" for item in combined_link)

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument("--headless=new")
options.binary_location = r"C:\Program Files\Google\Chrome\Application\chrome.exe"

# Set user agent and other headers to mimic a real browser
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.5672.63 Safari/537.36")
options.add_argument("accept-language=en-US,en;q=0.9")
options.add_argument("referer=https://www.google.com/")

# Disable features that indicate headless browsing
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option('excludeSwitches', ['enable-automation'])
options.add_experimental_option('useAutomationExtension', False)

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager(driver_version="125.0.6422.113").install()), options=options)

# Add delay between actions to mimic human interaction
def human_delay(): 
    time.sleep(int(2 + (random.random() * 3.5)))

# Create a CSV file to store the results
with open('scraped_data.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['link', 'text_content'])

    try:
        for url in articles_dict['urls']:
            driver.get(url)
            WebDriverWait(driver, timeout=10).until(lambda d: d.find_element(By.TAG_NAME, 'body'))
            human_delay()
            page_content = driver.find_element(By.TAG_NAME, 'body').text
            human_delay()
            
            # Write the URL and its content to the CSV file
            writer.writerow([url, page_content])

            print(page_content)
    finally:
        driver.quit()

In [8]:
from selenium.common.exceptions import TimeoutException

options = webdriver.ChromeOptions()
options.add_argument("--headless=new")
options.binary_location = r"C:\Program Files\Google\Chrome\Application\chrome.exe"

# Set user agent and other headers to mimic a real browser
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.5672.63 Safari/537.36")
options.add_argument("accept-language=en-US,en;q=0.9")
options.add_argument("referer=https://www.google.com/")

# Disable features that indicate headless browsing
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option('excludeSwitches', ['enable-automation'])
options.add_experimental_option('useAutomationExtension', False)

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager(driver_version="125.0.6422.113").install()), options=options)

# Set the page load timeout to 30 seconds  
driver.set_page_load_timeout(30)

# Create a CSV file to store the results
with open('scraped_data.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['link', 'text_content'])

    try:
        for url in articles_dict['urls']:
            try:
                driver.get(url)
                WebDriverWait(driver, timeout=10).until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
                human_delay()
                page_content = driver.find_element(By.TAG_NAME, 'body').text
                human_delay()
                
                # Write the URL and its content to the CSV file
                writer.writerow([url, page_content])

                print(page_content)
            except TimeoutException:
                print(f"Timeout occurred for URL: {url}")
    finally:
        driver.quit()

Konten Premium
E-Paper
BisnisIndonesia.id
Bisnismuda.id
Konten Interaktif
Bisnis Plus
Bisnisgrafik
Tv.Bisnis.com
Hypeabis.id
Context.id
BREAKING NEWS
UCOK Jadi Terobosan Pemkab Badung Raih Universal Coverage Jamsostek
MARKET
Login
Nyaman tanpa iklan. Langganan BisnisPro
HOME
BURSA DAN SAHAM
OBLIGASI DAN REKSADANA
EMAS
KURS
KOMODITAS
REKOMENDASI
KORPORASI
Nyaman tanpa iklan. Langganan BisnisPro
Home Market Kurs
Nilai Tukar Rupiah terhadap Dolar AS Hari Ini, Kamis (30/5/2024)
Nilai tukar rupiah terhadap dolar AS diprediksi kembali melemah pada Kamis 30/5/2024) seiring dengan proyeksi The Fed menahan suku bunga acuan lebih lama.
Dionisio Damara Tonce - Bisnis.com
Kamis, 30 Mei 2024 | 15:06
Share
Perbesar
Nilai tukar rupiah terhadap dolar AS diprediksi kembali melemah pada Kamis 30/5/2024) seiring dengan proyeksi The Fed menahan suku bunga acuan lebih lama. Bisnis/Suselo Jati
Smallest Font
Largest Font
LIVE TIMELINE
15:04 WIB
14:27 WIB
12:12 WIB
10:10 WIB
09:07 WIB
Bisnis.com, JAKARTA – Ni

TimeoutException: Message: timeout: Timed out receiving message from renderer: -0.002
  (Session info: chrome=125.0.6422.141)
Stacktrace:
	GetHandleVerifier [0x00B8B8E3+45827]
	(No symbol) [0x00B1DCC4]
	(No symbol) [0x00A1150F]
	(No symbol) [0x009FD496]
	(No symbol) [0x009FD311]
	(No symbol) [0x009FB739]
	(No symbol) [0x009FBD1D]
	(No symbol) [0x00A0802A]
	(No symbol) [0x00A1BF01]
	(No symbol) [0x00A203A6]
	(No symbol) [0x009FC38C]
	(No symbol) [0x00A1BCE3]
	(No symbol) [0x00A8BF53]
	(No symbol) [0x00A72B96]
	(No symbol) [0x00A46998]
	(No symbol) [0x00A4751D]
	GetHandleVerifier [0x00E44513+2899763]
	GetHandleVerifier [0x00E9793D+3240797]
	GetHandleVerifier [0x00C113B4+593364]
	GetHandleVerifier [0x00C182DC+621820]
	(No symbol) [0x00B270A4]
	(No symbol) [0x00B237A8]
	(No symbol) [0x00B23947]
	(No symbol) [0x00B159FE]
	BaseThreadInitThunk [0x76E5FCC9+25]
	RtlGetAppContainerNamedObjectPath [0x77D27C5E+286]
	RtlGetAppContainerNamedObjectPath [0x77D27C2E+238]


In [ ]:
# Extract articles from all_articles dictionary and store in articles list
articles = all_articles["articles"]

# Create empty lists to store URLs and content
urls = []
titles = []
news_contents = []
dates = []

# Iterate through each article and retrieve URL and content, appending URL to url_only list
for article in articles:
  url = article.get('url')
  news = article.get('content')
  date = article.get('publishedAt')
  news_contents.append(news)
  dates.append(date)
  urls.append(url)

In [ ]:
# List to collect dictionaries of each row
rows = []

# Loop through each URL and scrape the text content
for url, date in zip(urls, dates):
    # Make a request to the URL
    r = requests.get(url)

    # Check if the request was successful
    if r.status_code == 200:
        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(r.content, 'html.parser')

        # Extract the text content
        text = soup.get_text()

        # Add the URL, date, and text content to the list as a dictionary
        rows.append({'url': url, 'date': date, 'text': text})
    else:
        # If the request was not successful, add NaN values
        print('Error: Could not retrieve content from', url)
        rows.append({'url': url, 'date': date, 'text': np.nan})

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(rows)

# Write the DataFrame to a CSV file
df.to_csv('/content/news.csv', index=False)

In [ ]:
data = pd.read_csv('/content/news.csv')
data = data.dropna()
data.shape

In [ ]:
text_list = data.iloc[:, 2]
stop_words = set(stopwords.words('english'))

def clean_text(text):
    # Remove any URLs
    text = re.sub(r"http\S+", "", text)
    # Remove any non-alphanumeric characters
    text = re.sub(r"[^\w\s]", "", text)
    text = re.sub(r"\n", "", text)
    text = re.sub(r"\s+", " ", text)
    text = re.sub(r"(?<=[a-z])(?=[A-Z])", " ", text)
    text = text.lower()
    # Remove stop words
    text = " ".join([word for word in text.split() if word not in stop_words])
    return text

# Clean the text column
text_list_clean = text_list.apply(clean_text)

In [ ]:
from transformers import TextClassificationPipeline, AutoModelForSequenceClassification, AutoTokenizer

text_list_clean = list(text_list_clean)

model_name = "ElKulako/cryptobert"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels = 3)
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, max_length=500, truncation=True, padding = 'max_length')

preds = pipe(text_list_clean)
print(preds)

In [ ]:
sentiment_label = []
sentiment_values = []

for data in preds:
  label = data.get('label')
  value = data.get('score')
  sentiment_label.append(label)
  sentiment_values.append(value)

In [ ]:
df['text_clean'] = text_list_clean
df['sentiment_value'] = sentiment_values
df['sentiment_labels'] = sentiment_label

In [ ]:
df

In [ ]:
df.to_csv('/content/bitcoin_news.csv', index=False)

##Sentiment analysis using longformer BERT

In [ ]:
# # Load the pre-trained BERT model and tokenizer
# model_name = 'allenai/longformer-base-4096'
# tokenizer = transformers.LongformerTokenizer.from_pretrained(model_name)
# model = transformers.LongformerForSequenceClassification.from_pretrained(model_name)

# max_length = 4096
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model.to(device)
# sentiment_values = []

# for article in text_list_clean:
#     text = article
#     if text is None:
#         continue
#     tokens = tokenizer.encode_plus(text, max_length=max_length, padding='max_length', truncation=True, return_tensors='pt')
#     input_ids = tokens['input_ids'].to(device)
#     attention_mask = tokens['attention_mask'].to(device)
#     with torch.no_grad():
#         outputs = model(input_ids, attention_mask=attention_mask)
#         logits = outputs[0]
#     sigmoid = torch.nn.Sigmoid()

#     pos_logit = logits[:, 1].squeeze()
#     neg_logit = logits[:, 0].squeeze()
#     diff = pos_logit - neg_logit
#     sentiment_score = torch.sigmoid(diff).item()

#     print("Sentiment score:", sentiment_score)
#     sentiment_values.append(sentiment_score)